# Year to date homicide comparison, 2015 to 2016

By the end of August, 2016, it appears that the number of homicides for 2016 will have surpassed the number of homicides for all of 2015.

This is a comparison of the homicides, broken down by age of victim, geography and some temporal aspects, through August 31, comparing 2015 to 2016.

In [21]:
# Load some data

import os
import requests

def get_table_url(table_name, base_url=os.environ['NEWSROOMDB_URL']):
    return '{}table/json/{}'.format(os.environ['NEWSROOMDB_URL'], table_name)

def get_table_data(table_name):
    url = get_table_url(table_name)
    
    try:
        r = requests.get(url)
        return r.json()
    except:
        print("Request failed. Probably because the response is huge.  We should fix this.")
        return get_table_data(table_name)
    
homicides_raw = get_table_data('homicides')

In [22]:
# Pull homicides into a dataframe

import pandas as pd

homicides = pd.DataFrame(homicides_raw)

## Filter to year-to-day, 2015/2016 homicides only

In [23]:
# Add a column that contains date and time together and is a Python datetime
from datetime import datetime

def get_datetime(row):
    if not row['Occ Date']:
        return None
    
    dt_str = row['Occ Date']
    fmt_str = "%Y-%m-%d"
    if row['Occ Time']:
        dt_str = dt_str + " " + row['Occ Time']
        fmt_str += " %H:%M"
        
    return datetime.strptime(dt_str, fmt_str)

def get_month(row):
    return row['datetime'].month

homicides['datetime'] = homicides.apply(get_datetime, axis=1)
homicides['year'] = homicides.apply(lambda row: row['datetime'].year, axis=1)
homicides['month'] = homicides.apply(get_month, axis=1)

In [24]:
# Filter to homicides on or before this day

from datetime import datetime

# If we wanted to do this to the exact day of the year, which accomodates the fact that 2016 was a leap year,
# we would do it this way, however, I think it will be easier to talk to editors about it if we
# do a straight-up comparison through 8/31.

# day_of_year = datetime.now().timetuple().tm_yday
#
#def get_day_of_year(row):
#    try:
#        return row['datetime'].timetuple().tm_yday
#    except ValueError:
#        return None
#
#homicides['day_of_year'] = homicides.apply(get_day_of_year, axis=1)
#
#homicides_with_day_of_year = homicides[homicides['day_of_year'] != None]

end_date = datetime(2016, 9, 1)

def is_before_date(row):
    if row['datetime'].year == 2016:
        return row['datetime'] < end_date
    elif row['datetime'].year == 2015:
        return row['datetime'] < end_date.replace(year=2015) 
    else:
        return False
    
homicides_to_day = homicides[homicides.apply(is_before_date, axis=1)]

In [25]:
# Normalize district numbers

def normalize_district_number(d):
    try:
        return int(d)
    except ValueError:
        return d

homicides_to_day['District Number'] = homicides_to_day['District Number'].apply(normalize_district_number)

/Users/ghing/venvs/public-notebooks/lib/python3.4/site-packages/ipykernel/__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [26]:
# Get only 2015, 2016 homicides

homicides_to_day_2015 = homicides_to_day[homicides_to_day['year'] == 2015]
homicides_to_day_2016 = homicides_to_day[homicides_to_day['year'] == 2016]

## 2015 homicides

In [27]:
from IPython.display import display

with pd.option_context('display.max_rows', 2000):
    display(homicides_to_day_2015[['datetime', 'Age', 'District Number']].sort_values('datetime'))

,datetime,Age,District Number
648,2015-01-03 01:30:00,39,15
651,2015-01-04 20:50:00,20,10
647,2015-01-05 21:44:00,56,8
646,2015-01-05 23:40:00,24,6
649,2015-01-06 15:30:00,19,4
650,2015-01-06 15:30:00,18,4
667,2015-01-06 22:20:00,1,7
652,2015-01-08 20:45:00,51,10
1090,2015-01-09 22:40:00,35,11
653,2015-01-13 17:00:00,21,9


## 2016 homicides

In [28]:
from IPython.display import display

with pd.option_context('display.max_rows', 2000):
    display(homicides_to_day_2016[['datetime', 'Age', 'District Number']].sort_values('datetime'))

,datetime,Age,District Number
926,2016-01-01 02:00:00,24,2
1506,2016-01-01 06:40:00,36,9
1599,2016-01-01 21:40:00,32,7
1503,2016-01-02 16:45:00,16,3
1536,2016-01-04 08:20:00,21,15
1504,2016-01-04 19:35:00,33,12
1531,2016-01-05 15:35:00,16,4
1530,2016-01-05 15:35:00,17,4
927,2016-01-05 20:20:00,25,9
1505,2016-01-05 20:20:00,23,9


## Median Age

In [29]:
from IPython.display import display
import numpy as np
import pandas as pd

median_age_2015 = homicides_to_day_2015['Age'][homicides_to_day_2015['Age'].apply(np.isreal)].median()
median_age_2016 = homicides_to_day_2016['Age'][homicides_to_day_2016['Age'].apply(np.isreal)].median()

median_age = pd.DataFrame({
    2015: pd.Series([median_age_2015], index=['Median age']),
    2016: pd.Series([median_age_2016], index=['Median age']),   
})
display(median_age)


,2015,2016
Median age,25.0,25.5


## Oldest/youngest

In [30]:
from IPython.display import display
import numpy as np
import pandas as pd

youngest_2015 = homicides_to_day_2015['Age'][homicides_to_day_2015['Age'].apply(np.isreal)].min()
youngest_2016 = homicides_to_day_2016['Age'][homicides_to_day_2016['Age'].apply(np.isreal)].min()
oldest_2015 = homicides_to_day_2015['Age'][homicides_to_day_2015['Age'].apply(np.isreal)].max()
oldest_2016 = homicides_to_day_2016['Age'][homicides_to_day_2016['Age'].apply(np.isreal)].max()

oldest_youngest = pd.DataFrame({
    2015: pd.Series([youngest_2015, oldest_2015], index=['Youngest victim age', 'Oldest victim age']), 
    2016: pd.Series([youngest_2016, oldest_2016], index=['Youngest victim age', 'Oldest victim age']),         
})
display(oldest_youngest)

,2015,2016
Youngest victim age,1,2
Oldest victim age,79,84


## By police district

In [31]:
from IPython.display import display

by_police_district = pd.DataFrame({
    2015: homicides_to_day_2015.groupby('District Number').size(),
    2016: homicides_to_day_2016.groupby('District Number').size(),
})

by_police_district['change'] = by_police_district[2016] - by_police_district[2015]
by_police_district['percent change'] = (((by_police_district[2016] - by_police_district[2015]) / by_police_district[2015]) * 100)

display(by_police_district)

,2015,2016,change,percent change
1,4.0,8,4.0,100.000000
2,14.0,20,6.0,42.857143
3,15.0,29,14.0,93.333333
4,26.0,21,-5.0,-19.230769
5,19.0,23,4.0,21.052632
6,22.0,35,13.0,59.090909
7,25.0,55,30.0,120.000000
8,19.0,41,22.0,115.789474
9,20.0,33,13.0,65.000000
10,22.0,25,3.0,13.636364


## By month

In [32]:
from IPython.display import display

by_month = pd.DataFrame({
    2015: homicides_to_day_2015.groupby('month').size(),
    2016: homicides_to_day_2016.groupby('month').size(),
})

by_month['change'] = by_month[2016] - by_month[2015]
by_month['percent change'] = (((by_month[2016] - by_month[2015]) / by_month[2015]) * 100)

display(by_month)

,2015,2016,change,percent change
month,,,,
1.0,32,57,25,78.125000
2.0,20,45,25,125.000000
3.0,35,49,14,40.000000
4.0,35,41,6,17.142857
5.0,49,68,19,38.775510
6.0,49,72,23,46.938776
7.0,54,71,17,31.481481
8.0,57,91,34,59.649123


## By weekend

In [33]:
from datetime import datetime, timedelta
from dateutil import rrule
from dateutil import parser
from dateutil import relativedelta
from IPython.display import display

today = datetime.now().date()
fridays_2015 = rrule.rrule(rrule.WEEKLY, byweekday=relativedelta.FR(1), dtstart=datetime(2015,1,1), count=52)
fridays_2016 = rrule.rrule(rrule.WEEKLY, byweekday=relativedelta.FR(1), dtstart=datetime(2016,1,1), count=52)

end_date_2016 = datetime(2016, 9, 1)
end_date_2015 = end_date.replace(year=2015)
fridays_2015_to_date = [d.date() for d in fridays_2015 if d < end_date_2015]
fridays_2016_to_date = [d.date() for d in fridays_2016 if d < end_date_2016]

def get_weekends(fridays):
    weekends = []
    for friday in fridays:
        friday_dt = datetime(year=friday.year, month=friday.month, day=friday.day, hour=15, minute=0)
        monday_dt = (friday_dt + timedelta(days=3)).replace(hour=6, minute=0)
        weekends.append((friday_dt, monday_dt))
        
    return weekends
    
weekends_2015_to_date = get_weekends(fridays_2015_to_date)
weekends_2016_to_date = get_weekends(fridays_2016_to_date)

def is_weekend(timestamp):
    """Does the timestamp fall between Friday 3 p.m. and Monday 6 a.m."""
    if not timestamp:
        return False
    
    day_of_week = timestamp.weekday()
    
    if day_of_week > 0 and day_of_week < 4:
        return False
    
    if day_of_week == 4:
        # Friday
        
        # Same day, 3 p.m.
        start = datetime(timestamp.year, timestamp.month, timestamp.day, 15)
        
        return timestamp >= start
    
    if day_of_week == 0:
        # Monday
        
        # Same day, 6 a.m.
        end = datetime(timestamp.year, timestamp.month, timestamp.day, 6)
        
        return timestamp < end
        
    return True

homicides_to_day_2015_weekend = homicides_to_day_2015[homicides_to_day_2015.apply(lambda r: is_weekend(r['datetime']), axis=1)]
homicides_to_day_2016_weekend = homicides_to_day_2016[homicides_to_day_2016.apply(lambda r: is_weekend(r['datetime']), axis=1)]

def get_weekend_2015(row):
    for i, weekend in enumerate(weekends_2015_to_date):
        weekend_start, weekend_end = weekend
        if row['datetime'] >= weekend_start and row['datetime'] <= weekend_end:
            return i + 1
        
def get_weekend_2016(row):
    for i, weekend in enumerate(weekends_2016_to_date):
        weekend_start, weekend_end = weekend
        if row['datetime'] >= weekend_start and row['datetime'] <= weekend_end:
            return i + 1
        
homicides_to_day_2015_weekend['weekend'] = homicides_to_day_2015_weekend.apply(get_weekend_2015, axis=1)
homicides_to_day_2016_weekend['weekend'] = homicides_to_day_2016_weekend.apply(get_weekend_2016, axis=1)

by_weekend = pd.DataFrame({
    2015: homicides_to_day_2015_weekend.groupby('weekend').size(),
    2016: homicides_to_day_2016_weekend.groupby('weekend').size(),
})

by_weekend['change'] = by_weekend[2016] - by_weekend[2015]
by_weekend['percent change'] = (((by_weekend[2016] - by_weekend[2015]) / by_weekend[2015]) * 100)

display(by_weekend)

/Users/ghing/venvs/public-notebooks/lib/python3.4/site-packages/ipykernel/__main__.py:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/ghing/venvs/public-notebooks/lib/python3.4/site-packages/ipykernel/__main__.py:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,2015,2016,change,percent change
weekend,,,,
1,2.0,2,0.0,0.000000
2,1.0,7,6.0,600.000000
3,4.0,4,0.0,0.000000
4,5.0,5,0.0,0.000000
5,4.0,9,5.0,125.000000
6,2.0,4,2.0,100.000000
7,2.0,7,5.0,250.000000
8,3.0,5,2.0,66.666667
9,NaN,2,NaN,NaN
